In [1]:
%cd ../..

D:\Playground\AdvancedTimeSeriesForecastingBook\Github\Modern-Time-Series-Forecasting-with-Python-


In [2]:
import numpy as np
import os
import pandas as pd
from pathlib import Path
from tqdm.autonotebook import tqdm
import statsmodels.api as sm
import warnings
np.random.seed()
tqdm.pandas()

C:\Users\MANUJO~1\AppData\Local\Temp/ipykernel_2508/2442497750.py:5: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


# Reading Blocks 0-7

In [3]:
from src.utils.data_utils import compact_to_expanded

In [4]:
block_df = pd.read_parquet("data/london_smart_meters/preprocessed/london_smart_meters_merged_block_0-7.parquet")

block_df.head()

,LCLid,start_timestamp,frequency,energy_consumption,series_length,stdorToU,Acorn,Acorn_grouped,file,holidays,...,windBearing,temperature,dewPoint,pressure,apparentTemperature,windSpeed,precipType,icon,humidity,summary
0,MAC000002,2012-10-13,30min,"[0.263, 0.2689999999999999, 0.275, 0.256, 0.21...",24144,Std,ACORN-A,Affluent,block_0,"[NO_HOLIDAY, NO_HOLIDAY, NO_HOLIDAY, NO_HOLIDA...",...,"[186, 186, 188, 188, 190, 190, 203, 203, 206, ...","[8.78, 8.78, 8.27, 8.27, 7.87, 7.87, 7.89, 7.8...","[6.28, 6.28, 6.21, 6.21, 6.22, 6.22, 6.76, 6.7...","[1007.7, 1007.7, 1007.36, 1007.36, 1006.73, 10...","[7.55, 7.55, 7.34, 7.34, 6.75, 6.75, 6.89, 6.8...","[2.28, 2.28, 1.81, 1.81, 1.95, 1.95, 1.83, 1.8...","[rain, rain, rain, rain, rain, rain, rain, rai...","[clear-night, clear-night, clear-night, clear-...","[0.84, 0.84, 0.87, 0.87, 0.89, 0.89, 0.93, 0.9...","[Clear, Clear, Clear, Clear, Partly Cloudy, Pa..."
1,MAC000246,2012-01-01,30min,"[0.509, 0.317, 0.253, 0.249, 0.93, 0.607, 0.10...",37872,Std,ACORN-A,Affluent,block_0,"[NO_HOLIDAY, NO_HOLIDAY, NO_HOLIDAY, NO_HOLIDA...",...,"[229, 229, 238, 238, 229, 229, 231, 231, 227, ...","[12.12, 12.12, 12.59, 12.59, 12.45, 12.45, 12....","[10.97, 10.97, 11.02, 11.02, 11.04, 11.04, 10....","[1008.1, 1008.1, 1007.88, 1007.88, 1007.95, 10...","[12.12, 12.12, 12.59, 12.59, 12.45, 12.45, 12....","[5.9, 5.9, 6.06, 6.06, 5.31, 5.31, 4.68, 4.68,...","[rain, rain, rain, rain, rain, rain, rain, rai...","[partly-cloudy-night, partly-cloudy-night, clo...","[0.93, 0.93, 0.9, 0.9, 0.91, 0.91, 0.93, 0.93,...","[Mostly Cloudy, Mostly Cloudy, Overcast, Overc..."
2,MAC000450,2012-03-23,30min,"[1.337, 1.426, 0.996, 0.971, 0.994, 0.952, 0.8...",33936,Std,ACORN-A,Affluent,block_0,"[NO_HOLIDAY, NO_HOLIDAY, NO_HOLIDAY, NO_HOLIDA...",...,"[78, 78, 73, 73, 81, 81, 80, 80, 75, 75, 71, 7...","[8.76, 8.76, 8.54, 8.54, 8.09, 8.09, 7.34, 7.3...","[7.25, 7.25, 7.12, 7.12, 7.17, 7.17, 6.68, 6.6...","[1027.41, 1027.41, 1026.91, 1026.91, 1026.54, ...","[7.59, 7.59, 7.43, 7.43, 7.24, 7.24, 7.34, 7.3...","[2.18, 2.18, 2.07, 2.07, 1.72, 1.72, 1.34, 1.3...","[rain, rain, rain, rain, rain, rain, rain, rai...","[fog, fog, fog, fog, fog, fog, fog, fog, fog, ...","[0.9, 0.9, 0.91, 0.91, 0.94, 0.94, 0.96, 0.96,...","[Foggy, Foggy, Foggy, Foggy, Foggy, Foggy, Fog..."
3,MAC001074,2012-05-09,30min,"[0.18, 0.086, 0.106, 0.173, 0.146, 0.223, 0.21...",31680,ToU,ACORN-,ACORN-,block_0,"[NO_HOLIDAY, NO_HOLIDAY, NO_HOLIDAY, NO_HOLIDA...",...,"[215, 215, 207, 207, 215, 215, 216, 216, 126, ...","[11.46, 11.46, 11.38, 11.38, 11.38, 11.38, 10....","[10.23, 10.23, 10.17, 10.17, 10.24, 10.24, 10....","[1007.39, 1007.39, 1007.21, 1007.21, 1007.06, ...","[11.46, 11.46, 11.38, 11.38, 11.38, 11.38, 10....","[2.35, 2.35, 2.15, 2.15, 1.84, 1.84, 1.22, 1.2...","[rain, rain, rain, rain, rain, rain, rain, rai...","[partly-cloudy-night, partly-cloudy-night, par...","[0.92, 0.92, 0.92, 0.92, 0.93, 0.93, 0.95, 0.9...","[Partly Cloudy, Partly Cloudy, Mostly Cloudy, ..."
4,MAC003223,2012-09-18,30min,"[0.076, 0.079, 0.123, 0.109, 0.051, 0.069, 0.0...",25344,Std,ACORN-A,Affluent,block_0,"[NO_HOLIDAY, NO_HOLIDAY, NO_HOLIDAY, NO_HOLIDA...",...,"[236, 236, 240, 240, 242, 242, 244, 244, 248, ...","[14.06, 14.06, 13.12, 13.12, 12.53, 12.53, 12....","[10.82, 10.82, 10.29, 10.29, 9.86, 9.86, 9.83,...","[1011.09, 1011.09, 1010.82, 1010.82, 1010.65, ...","[14.06, 14.06, 13.12, 13.12, 12.53, 12.53, 12....","[3.86, 3.86, 3.81, 3.81, 4.27, 4.27, 4.12, 4.1...","[rain, rain, rain, rain, rain, rain, rain, rai...","[clear-night, clear-night, clear-night, clear-...","[0.81, 0.81, 0.83, 0.83, 0.84, 0.84, 0.86, 0.8...","[Clear, Clear, Clear, Clear, Clear, Clear, Par..."


Let's take all the consumers in one block for our exercise. Keep in mind, you can do the same with more number of blocks as long as it fits in your RAM

In [5]:
# uncomment and further filter to a single block if you have RAM constraints
# block_df = block_df[block_df.file=="block_7"]

In [6]:
#Converting to expanded form
exp_block_df = compact_to_expanded(block_df, timeseries_col = 'energy_consumption',
static_cols = ["frequency", "series_length", "stdorToU", "Acorn", "Acorn_grouped", "file"],
time_varying_cols = ['holidays', 'visibility', 'windBearing', 'temperature', 'dewPoint',
       'pressure', 'apparentTemperature', 'windSpeed', 'precipType', 'icon',
       'humidity', 'summary'],
ts_identifier = "LCLid")

exp_block_df.head()

  0%|          | 0/399 [00:00<?, ?it/s]

,timestamp,LCLid,energy_consumption,frequency,series_length,stdorToU,Acorn,Acorn_grouped,file,holidays,...,windBearing,temperature,dewPoint,pressure,apparentTemperature,windSpeed,precipType,icon,humidity,summary
0,2012-10-13 00:00:00,MAC000002,0.263,30min,24144,Std,ACORN-A,Affluent,block_0,NO_HOLIDAY,...,186,8.78,6.28,1007.70,7.55,2.28,rain,clear-night,0.84,Clear
1,2012-10-13 00:30:00,MAC000002,0.269,30min,24144,Std,ACORN-A,Affluent,block_0,NO_HOLIDAY,...,186,8.78,6.28,1007.70,7.55,2.28,rain,clear-night,0.84,Clear
2,2012-10-13 01:00:00,MAC000002,0.275,30min,24144,Std,ACORN-A,Affluent,block_0,NO_HOLIDAY,...,188,8.27,6.21,1007.36,7.34,1.81,rain,clear-night,0.87,Clear
3,2012-10-13 01:30:00,MAC000002,0.256,30min,24144,Std,ACORN-A,Affluent,block_0,NO_HOLIDAY,...,188,8.27,6.21,1007.36,7.34,1.81,rain,clear-night,0.87,Clear
4,2012-10-13 02:00:00,MAC000002,0.211,30min,24144,Std,ACORN-A,Affluent,block_0,NO_HOLIDAY,...,190,7.87,6.22,1006.73,6.75,1.95,rain,partly-cloudy-night,0.89,Partly Cloudy


## Reduce Memory Footprint

In [7]:
from src.utils.data_utils import reduce_memory_footprint

In [8]:
exp_block_df.info(memory_usage="deep", verbose=False)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11821680 entries, 0 to 35279
Columns: 21 entries, timestamp to summary
dtypes: datetime64[ns](1), float64(8), int64(2), object(10)
memory usage: 8.2 GB


In [9]:
exp_block_df = reduce_memory_footprint(exp_block_df)

In [10]:
exp_block_df.info(memory_usage="deep", verbose=False)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11821680 entries, 0 to 35279
Columns: 21 entries, timestamp to summary
dtypes: category(10), datetime64[ns](1), float32(8), int32(2)
memory usage: 755.4 MB


# Train Test Valildation Split

We are going to keep 2014 data as the validation and test period. We have 2 months(Jan and Feb) of data in 2014. Jan is Validation and Feb is Test

In [16]:
test_mask = (exp_block_df.timestamp.dt.year==2014) & (exp_block_df.timestamp.dt.month==2)
val_mask = (exp_block_df.timestamp.dt.year==2014) & (exp_block_df.timestamp.dt.month==1)

train = exp_block_df[~(val_mask|test_mask)]
val = exp_block_df[val_mask]
test = exp_block_df[test_mask]
print(f"# of Training samples: {len(train)} | # of Validation samples: {len(val)} | # of Test samples: {len(test)}")
print(f"Max Date in Train: {train.timestamp.max()} | Min Date in Validation: {val.timestamp.min()} | Min Date in Test: {test.timestamp.min()}")

# of Training samples: 10710864 | # of Validation samples: 593712 | # of Test samples: 517104
Max Date in Train: 2013-12-31 23:30:00 | Min Date in Validation: 2014-01-01 00:00:00 | Min Date in Test: 2014-02-01 00:00:00


In [17]:
train.to_parquet("data/london_smart_meters/preprocessed/block_0-7_train.parquet")
val.to_parquet("data/london_smart_meters/preprocessed/block_0-7_val.parquet")
test.to_parquet("data/london_smart_meters/preprocessed/block_0-7_test.parquet")

## Train Test Split after filling in missing values

In [18]:
from src.imputation.interpolation import SeasonalInterpolation

block_df.energy_consumption = block_df.energy_consumption.progress_apply(lambda x: SeasonalInterpolation(seasonal_period=48*7).fit_transform(x.reshape(-1,1)).squeeze())

  0%|          | 0/399 [00:00<?, ?it/s]

D:\Playground\AdvancedTimeSeriesForecastingBook\Github\Modern-Time-Series-Forecasting-with-Python-\src\imputation\_solver.py:68: UserWarning: Input matrix is not missing any values
  warnings.warn("Input matrix is not missing any values")
D:\Playground\AdvancedTimeSeriesForecastingBook\Github\Modern-Time-Series-Forecasting-with-Python-\src\imputation\interpolation.py:83: UserWarning: [Seasonal Interpolation] Warning: provided matrix doesn't contain any missing values.
  warnings.warn(
D:\Playground\AdvancedTimeSeriesForecastingBook\Github\Modern-Time-Series-Forecasting-with-Python-\src\imputation\interpolation.py:86: UserWarning: [Seasonal Interpolation] The algorithm will run, but will return an unchanged matrix.
  warnings.warn(


In [19]:
#Converting to expanded form
exp_block_df = compact_to_expanded(block_df, timeseries_col = 'energy_consumption',
static_cols = ["frequency", "series_length", "stdorToU", "Acorn", "Acorn_grouped", "file"],
time_varying_cols = ['holidays', 'visibility', 'windBearing', 'temperature', 'dewPoint',
       'pressure', 'apparentTemperature', 'windSpeed', 'precipType', 'icon',
       'humidity', 'summary'],
ts_identifier = "LCLid")

exp_block_df.head()

  0%|          | 0/399 [00:00<?, ?it/s]

,timestamp,LCLid,energy_consumption,frequency,series_length,stdorToU,Acorn,Acorn_grouped,file,holidays,...,windBearing,temperature,dewPoint,pressure,apparentTemperature,windSpeed,precipType,icon,humidity,summary
0,2012-10-13 00:00:00,MAC000002,0.263,30min,24144,Std,ACORN-A,Affluent,block_0,NO_HOLIDAY,...,186,8.78,6.28,1007.70,7.55,2.28,rain,clear-night,0.84,Clear
1,2012-10-13 00:30:00,MAC000002,0.269,30min,24144,Std,ACORN-A,Affluent,block_0,NO_HOLIDAY,...,186,8.78,6.28,1007.70,7.55,2.28,rain,clear-night,0.84,Clear
2,2012-10-13 01:00:00,MAC000002,0.275,30min,24144,Std,ACORN-A,Affluent,block_0,NO_HOLIDAY,...,188,8.27,6.21,1007.36,7.34,1.81,rain,clear-night,0.87,Clear
3,2012-10-13 01:30:00,MAC000002,0.256,30min,24144,Std,ACORN-A,Affluent,block_0,NO_HOLIDAY,...,188,8.27,6.21,1007.36,7.34,1.81,rain,clear-night,0.87,Clear
4,2012-10-13 02:00:00,MAC000002,0.211,30min,24144,Std,ACORN-A,Affluent,block_0,NO_HOLIDAY,...,190,7.87,6.22,1006.73,6.75,1.95,rain,partly-cloudy-night,0.89,Partly Cloudy


## Reduce Memory Footprint

In [20]:
from src.utils.data_utils import reduce_memory_footprint

In [21]:
exp_block_df.info(memory_usage="deep", verbose=False)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11821680 entries, 0 to 35279
Columns: 21 entries, timestamp to summary
dtypes: datetime64[ns](1), float64(8), int64(2), object(10)
memory usage: 8.2 GB


In [22]:
exp_block_df = reduce_memory_footprint(exp_block_df)

In [23]:
exp_block_df.info(memory_usage="deep", verbose=False)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11821680 entries, 0 to 35279
Columns: 21 entries, timestamp to summary
dtypes: category(10), datetime64[ns](1), float32(8), int32(2)
memory usage: 755.4 MB


In [24]:
test_mask = (exp_block_df.timestamp.dt.year==2014) & (exp_block_df.timestamp.dt.month==2)
val_mask = (exp_block_df.timestamp.dt.year==2014) & (exp_block_df.timestamp.dt.month==1)

train = exp_block_df[~(val_mask|test_mask)]
val = exp_block_df[val_mask]
test = exp_block_df[test_mask]
print(f"# of Training samples: {len(train)} | # of Validation samples: {len(val)} | # of Test samples: {len(test)}")
print(f"Max Date in Train: {train.timestamp.max()} | Min Date in Validation: {val.timestamp.min()} | Min Date in Test: {test.timestamp.min()}")

# of Training samples: 10710864 | # of Validation samples: 593712 | # of Test samples: 517104
Max Date in Train: 2013-12-31 23:30:00 | Min Date in Validation: 2014-01-01 00:00:00 | Min Date in Test: 2014-02-01 00:00:00


In [25]:
train.to_parquet("data/london_smart_meters/preprocessed/block_0-7_train_missing_imputed.parquet")
val.to_parquet("data/london_smart_meters/preprocessed/block_0-7_val_missing_imputed.parquet")
test.to_parquet("data/london_smart_meters/preprocessed/block_0-7_test_missing_imputed.parquet")